In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
# Load dataset
anime = pd.read_csv("anime.csv")
anime

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [7]:
# Drop rows with missing names or genres
anime = anime.dropna(subset=["name", "genre"])
anime = anime.drop_duplicates(subset="name")
anime

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [8]:
import nltk

In [9]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [11]:
# Function to stem a sentence
def stem_text(text):
    words = nltk.word_tokenize(text.lower())
    stemmed_words = [ps.stem(word) for word in words]
    return " ".join(stemmed_words)

In [12]:
anime["stemmed_name"] = anime["name"].apply(stem_text)

In [13]:
# TF-IDF on genres
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(anime["genre"])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [14]:
# Function to get recommendations
def get_recommendations(input_title, cosine_sim=cosine_sim):
    stemmed_input = stem_text(input_title)
    match = anime[anime["stemmed_name"] == stemmed_input]
    
    if match.empty:
        return []
    
    idx = match.index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:11]
    anime_indices = [i[0] for i in sim_scores]
    return anime["name"].iloc[anime_indices].tolist()

In [15]:
# Input from user
if __name__ == "__main__":
    anime_title = input("Enter an anime title: ")
    recommendations = get_recommendations(anime_title)
    if recommendations:
        print(f"\nRecommendations for '{anime_title}':")
        for rec in recommendations:
            print("-", rec)
    else:
        print("Sorry, anime not found in the database.")

Enter an anime title: aftermath

Recommendations for 'aftermath':
- Omoide Poroporo
- Kimi ga Nozomu Eien
- Love Hina: Motoko no Sentaku, Koi ka Ken... &quot;Naku na&quot;
- Kimi no Iru Machi OVA
- Kimi no Iru Machi
- Lamune
- Kita e.: Diamond Dust Drops
- Akai Ito
- Bobby ni Kubittake
- Kizuna (Special)


In [28]:
# Streamlit UI
import streamlit as st
st.title("🎌 Anime Recommendation System")
user_input = st.text_input("Enter an anime title:")

if st.button("Recommend"):
    if user_input.strip() == "":
        st.warning("Please enter a valid anime title.")
    else:
        recommendations = get_recommendations(user_input)
        if recommendations:
            st.success("Top recommendations:")
            for rec in recommendations:
                st.write("👉", rec)
        else:
            st.error("Sorry, anime not found in the database.")

2025-04-20 22:28:47.580 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 22:28:48.012 
  command:

    streamlit run C:\Users\HP\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-04-20 22:28:48.015 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 22:28:48.017 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 22:28:48.018 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 22:28:48.020 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 22:28:48.022 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 22:28:48.024 Session state does not fun